In [21]:
import pandas as pd

In [44]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
with urlopen('https://fbref.com/en/matches/f13e982e/Real-Betis-Girona-August-15-2024-La-Liga') as response:
    soup = BeautifulSoup(response, 'html.parser')
    tables = soup.find_all("table")
    table_df = {}
    for i, table in enumerate(tables):
        table_id = f"No name table #{i}" if table.get("id") is None else table.get("id")
        table_df[table_id] = pd.read_html(str(table))[0]

C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\982059425.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table_df[table_id] = pd.read_html(str(table))[0]
C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\982059425.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table_df[table_id] = pd.read_html(str(table))[0]
C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\982059425.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table_df[table_id] = pd.read_html(str(table))[0]
C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\982059425.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be rem

In [52]:
# profile 
with urlopen('https://fbref.com/en/matches/f13e982e/Real-Betis-Girona-August-15-2024-La-Liga') as response:
    soup = BeautifulSoup(response, 'html.parser')
    tables = soup.find_all("table")
    table_df = {}
    for i, table in enumerate(tables):
        table_id = f"No name table #{i}" if table.get("id") is None else table.get("id")
        records = []
        columns = []
        for tr in table.findAll("tr"):
            ths = tr.findAll("th")
            if ths != []:
                for each in ths:
                    columns.append(each.text)
            else:
                trs = tr.findAll("td")
                record = []
                for each in trs:
                    try:
                        link = each.find('a')['href']
                        text = each.text
                        record.append(link)
                        record.append(text)
                    except:
                        text = each.text
                        record.append(text)
                records.append(record)

        columns.insert(1, 'Link')
        table_df[table_id] = pd.DataFrame(data=records)#, columns = columns)

In [50]:
pd.read_html(tables[3].prettify())

C:\Users\Huy's\AppData\Local\Temp\ipykernel_6956\2755705327.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(tables[3].prettify())


[   Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Player                  #             Nation   
 0        Aitor Ruibal               24.0            es  ESP   
 1      Ezequiel Ávila                9.0            ar  ARG   
 2              Juanmi                7.0            es  ESP   
 3     Abde Ezzalzouli               10.0            ma  MAR   
 4       Pablo Fornals               18.0            es  ESP   
 5               Rodri               17.0            es  ESP   
 6         Nabil Fekir                8.0            fr  FRA   
 7           Marc Roca               21.0            es  ESP   
 8      Sergi Altimira               16.0            es  ESP   
 9    William Carvalho               14.0            pt  POR   
 10     Johnny Cardoso                4.0            us  USA   
 11     Romain Perraud               15.0            fr  FRA   
 12     Diego Llorente                3.0            es  ESP   
 13        Marc Bartra                5.

In [54]:
table_df.keys()

dict_keys(['No name table #0', 'No name table #1', 'No name table #2', 'stats_fc536746_summary', 'stats_fc536746_passing', 'stats_fc536746_passing_types', 'stats_fc536746_defense', 'stats_fc536746_possession', 'stats_fc536746_misc', 'keeper_stats_fc536746', 'stats_9024a00a_summary', 'stats_9024a00a_passing', 'stats_9024a00a_passing_types', 'stats_9024a00a_defense', 'stats_9024a00a_possession', 'stats_9024a00a_misc', 'keeper_stats_9024a00a', 'shots_all', 'shots_fc536746', 'shots_9024a00a'])

In [55]:
table_df["stats_fc536746_summary"]

""
